### 0. 필요한 라이브러리 로드

In [1]:
import pymysql
from sqlalchemy import create_engine
# pip install mysqlclient

from abc import ABC, abstractmethod
from typing import *
import re

from prettytable import PrettyTable
import pandas as pd
pd.options.display.float_format = '{:.10f}'.format
import pprint

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 기존 클래스 (테이블 넣고 ~ 결합키, 결합대상정보 ~ 매핑테이블 만들기)

In [2]:
# import pymysql
# from abc import ABC, abstractmethod

class PreprocessQuery(ABC):
    """가명처리를 위한 개인정보 추출 목적의 SQL쿼리 추상클래스"""
    @abstractmethod
    def connectDatabase(self):
        """데이터베이스에 연결하기 위해 접속하는 메서드"""
        pass
    
    @abstractmethod
    def executeQuery(self, SQL):
        """SQL쿼리를 실행하는 메서드"""
        pass
    
    @abstractmethod
    def closeConnection(self):
        """데이터베이스와의 연결을 종료하는 메서드"""
        pass

In [3]:
# from pseudonymizer.encryptionPseudonyms.abstractpreprocessQuery import PreprocessQuery
# from typing import *

class PyMySQLQuery(PreprocessQuery):
    """MySQL Server데이터베이스에 연결하고 쿼리를 실행하여 데이터를 추출하는 클래스"""
    def __init__(self, pw):
        self._pw = pw
        self.DBconnection = ConnectMySQLserver(self._pw)
        self.SQL = None
    
    def connectDatabase(self, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        """MySQL DBMS 데이터베이스에 접속하는 메서드"""
        self.DBconnection.connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)
    
    def dataQueryLanguage(self, sql):
        """SQL쿼리문 작성 메서드(데이터 추출 쿼리문 캡슐화)"""
        self.SQL = f"{sql}"
    
    def executeQuery(self):
        """SQL쿼리문 실행 및 예외처리 메서드(데이터베이스로 쿼리를 보내서 실행)"""
        try:
            action_output = self.DBconnection.cursor.execute(self.SQL)
            return action_output
        except pymysql.Error as e:
            print(f"Error Executing Query: {e}")

    def useFetchallQuery(self):
        """SQL 쿼리 실행 결과의 cursor.fetchall() 을 사용할 수 있도록 하는 메서드"""
        try:
            action_output = self.DBconnection.cursor.execute(self.SQL)
            records = self.DBconnection.cursor.fetchall()
            return records
        except pymysql.Error as e:
            print(f"Executing query error: {e}")


    
    def commitTransaction(self):
        """실행결과를 확정(트랜잭션을 커밋)하는 메서드"""
        self.DBconnection.connection.commit()
    
    def closeConnection(self):
        """데이터베이스와의 연결을 종료하는 메서드"""
        self.DBconnection.close_connection()
        
    def executeQueryAsDataFrame(self):
        """SQL 쿼리를 실행한 결과를 판다스 데이터프레임으로 출력하는 메서드"""
        try:
            action_output = self.DBconnection.cursor.execute(self.SQL)
            records = self.DBconnection.cursor.fetchall()
            attributes = [i[0] for i in self.DBconnection.cursor.description]
            querydata = pd.DataFrame(records, columns = attributes)
            return querydata
        
        except pymysql.Error as e:
            print(f"Executing query error: {e}")

In [4]:
class ConnectMySQLserver:
    """데이터베이스 엔진을 연동하기 위한 접속 및 연결 종료 클래스"""
    def __init__(self, pw):
        self._pw = pw
        self.connection = None
        self.cursor = None
    
    def connectDatabase(self, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        """MySQL DBMS 데이터베이스에 접속 메서드
        : 서버IP주소, 사용자명, 계정 암호, 데이터베이스명, 한글 인코딩 방식"""
        try:
            self.connection = pymysql.connect(
                host=serverIP, port=port_num,
                user=user_name, password=self._pw,
                db=database_name, charset=kr_encoder
            )
            self.cursor = self.connection.cursor()
        except pymysql.Error as e:
            print(f"Error Connecting to MySQL from Python: {e}")
    
    def closeConnection(self):
        """연결 및 커서 닫기 메서드"""
        if self.cursor:
            self.cursor.close()
        if self.connection:
            self.connection.close()

In [5]:
class DivideMySQL(PyMySQLQuery):
    """테이블을 분할하여 DB에 저장하는 클래스"""
    def insertDB(self):
        pass

In [6]:
class JoinMySQL(PyMySQLQuery):
    """테이블 여러개를 Join하여 DB에 저장하는 클래스"""
    def joinDB(self):
        pass

In [7]:
class InsertKey(DivideMySQL):
    """결합키 생성항목 DB 분할 삽입 클래스"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        self.pw = pw
        super().__init__(pw = self.pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)


    def addSerialNum(self, data: str, serial_col: str, serial_text: str):
        """SQL로 원본 데이터에 일련번호 부여하기 메서드
           일련번호 형태를 테이블명+행번호가 기본, 혹은 table이 이름이면 t1, t2, t3 이렇게 진행되도록 하기

           * self.serial_col로 입력한 컬럼이 해당 테이블에 존재하는 경우 제거하고 작업하는 구문 삽입 필요
        """
        # super().dataQueryLanguage(f"ALTER TABLE {self.data} DROP COLUMN IF EXISTS {self.serial_col}")
        # super().executeQuery()
        
        # 컬럼 만들기
        make_column = f"ALTER TABLE {data} ADD COLUMN {serial_col} VARCHAR(1000)"
        super().dataQueryLanguage(make_column)
        super().executeQuery()

        # 값 할당
        super().dataQueryLanguage("SET @counter = 0;")
        super().executeQuery()
        
        super().dataQueryLanguage(f"UPDATE {data} SET {serial_col} = CONCAT('{serial_text}', @counter := @counter + 1);")
        super().executeQuery()

    def insertDB(self, data: str, schema_name: str, table_name: str, serial_col: str, columns: list, key_col: str):
        """결합키 생성 항목 columns를 받아 DB에 입력하는 메서드"""
        join_columns = ', '.join(columns)

        super().dataQueryLanguage(f"DROP TABLE IF EXISTS {schema_name}.{table_name}")
        super().executeQuery()
        
        sql = f"CREATE TABLE {schema_name}.{table_name} AS SELECT {serial_col}, {join_columns} FROM {data}"
        super().dataQueryLanguage(sql)
        super().executeQuery()

        # 결합키 컬럼 만들기
        super().dataQueryLanguage(f"ALTER TABLE {schema_name}.{table_name} ADD COLUMN {key_col} VARCHAR(1000)")
        super().executeQuery()

        super().dataQueryLanguage(f"UPDATE {schema_name}.{table_name} SET {key_col} = CONCAT({join_columns})")
        super().executeQuery()

    

In [8]:
class InsertTargets(DivideMySQL):
    """결합대상정보 DB 분할 삽입 클래스"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        self.pw = pw
        super().__init__(pw = self.pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)

    def insertDB(self, data: str, schema_name: str, table_name: str, columns: list):
        """결합대상정보 DB 입력 메서드
           : 원래 테이블을 복사한 뒤, 결합키 생성항목 columns를 제거하여 생성
        """
        super().dataQueryLanguage(f"DROP TABLE IF EXISTS {schema_name}.{table_name}")
        super().executeQuery()
        
        create_sql = f"CREATE TABLE {schema_name}.{table_name} AS SELECT * FROM {data}"
        super().dataQueryLanguage(create_sql)
        super().executeQuery()

        for column in columns:
            drop_sql = f"ALTER TABLE {schema_name}.{table_name} drop {column}"
            super().dataQueryLanguage(drop_sql)
            super().executeQuery()

In [10]:
import os
import binascii

class CreateKeyCol(PyMySQLQuery):
    """SALT값을 더해 결합키를 암호화하는 클래스"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        self.pw = pw
        self.kr_encoder = kr_encoder
        super().__init__(pw = self.pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)

    def createKey(self, func: str, schema: str, table: str, key_col: str, salt_col: str):
        """결합키 암호화 방식을 선택하여 실행시키는 메서드"""
        if func == "SHA256":
            self.applySHA256(schema, table, salt_col, key_col)
        elif func == "SHA512":
            self.applySHA512(schema, table, salt_col, key_col)

    def createSalt(self, schema: str, table: str, salt_col: str, key_col: str):
        """SALT값을 만들어 테이블 특정 컬럼에 붙이는 메서드"""
        # SALT값 컬럼 만들기
        super().dataQueryLanguage(f"ALTER TABLE {salt_col} VARCHAR(1000)")
        super().executeQuery()

        # SALT값을 만들고 컬럼에 입력하기
        super().dataQueryLanguage(f"SELECT * FROM {schema}.{table}")
        rows = super().useFetchallQuery()

        for row in rows:
            salt = binascii.hexlify(os.urandom(16)).decode(self.kr_encoder)
            sql = f"UPDATE {schema}.{table} SET {salt_col} = {salt} WHERE {key_col} = {row[0]}"
            super().dataQueryLanguage(sql)
            super().executeQuery()

            
    def applySHA256(self, schema: str, table: str, key_col: str, salt_col: str):
        """SHA256 해시함수를 통해 결합키 컬럼을 암호화하는 메서드"""
        sql = f"UPDATE {schema}.{table} SET {key_col} = SHA2(CONCAT({key_col}, {salt_col}), 256)"
        super().dataQueryLanguage(sql)
        super().executeQuery()

    def applySHA512(self, schema: str, table: str, key_col: str, salt_col: str):
        """SHA512 해시함수를 통해 결합키 컬럼을 암호화하는 메서드"""
        sql = f"UPDATE {schema}.{table} SET {key_col} = SHA2(CONCAT({key_col}, {salt_col}), 512)"
        super().dataQueryLanguage(sql)
        super().executeQuery()


In [11]:
class CreateMappingTable(JoinMySQL):
    """결합키연계정보, 일련번호, 결합키, SALT 등 매핑테이블 만드는 클래스"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        self.pw = pw
        self._createKeyCol = None
        super().__init__(pw = self.pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)

    def addCreateKeyCol(self, createKeyCol: CreateKeyCol):
        self._createKeyCol = createKeyCol

    def createCryptoKey(self, schemas: list, tables: list, func: str, salt_col: str, key_col: str):
        """SALT값을 만든 뒤 결합키를 암호화하는 메서드"""
        for i, table in enumerate(tables):
            self._createKeyCol.createSalt(schemas[i], table, salt_col, key_col)
            self._createKeyCol.createKey(schemas[i], func, table, salt_col, key_col)

    def renameCol(self, schemas: list, tables: list, serial_col: str, suffixes: list):
        """테이블별로 serial_col 이었던 컬럼명을 {serial_col}_{suffixes[i]} 형태로 바꾸는 메서드"""
        for i in range(len(tables)):
            super().dataQueryLanguage(f"ALTER TABLE {schemas[i]}.{tables[i]} RENAME COLUMN {serial_col} to {serial_col}_{suffixes[i]}")
            super().executeQuery()

    def joinDB(self, schemas: list, tables: list, serial_col: str, suffixes: list, result_schema: str, result_table: str, key_col: str):
        """일련번호를 결합키 기준으로 결합하여 매핑테이블 만드는 메서드

           예시 구문 : 
           CREATE table joined_view2 AS
           SELECT serialnum_2, serialnum_3, new_table_concat_r.result 
           FROM new_table_concat_r
           INNER JOIN new_table_concat_2 ON new_table_concat_r.result = new_table_concat_2.result;
           
           
        """
        super().dataQueryLanguage(f"DROP TABLE IF EXISTS {result_schema}.{result_table}")
        super().executeQuery()

        create_sql = f"CREATE TABLE {result_schema}.{result_table} AS "
        select_sql = f"SELECT {schemas[0]}.{tables[0]}.{key_col}, {schemas[0]}.{tables[0]}.{serial_col}_{suffixes[0]}, {schemas[1]}.{tables[1]}.{serial_col}_{suffixes[1]}  "
        from_sql = f"FROM {schemas[0]}.{tables[0]} "
        join_sql = f"INNER JOIN {schemas[1]}.{tables[1]} ON {schemas[0]}.{tables[0]}.{key_col} = {schemas[1]}.{tables[1]}.{key_col} "

        if len(tables) > 2:
            for i in range(2, len(tables)):
                select_sql += f"{serial_col}_{suffixes[i]}, "
                join_sql += f"INNER JOIN {schemas[i]}.{tables[i]} ON {schemas[0]}.{tables[0]}.{key_col} = {schemas[i]}.{tables[i]}.{key_col} "
        else:
            pass

        select_sql = select_sql[:-2] + " "

        sql = create_sql + select_sql + from_sql + join_sql

        super().dataQueryLanguage(sql)
        super().executeQuery()

In [12]:
from abc import ABC, abstractmethod

## 새 클래스 과정 1 : 원본 테이블을 결합키와 결합대상정보로 분할하여 매핑테이블 만들기
- 스키마-테이블 저장 및 조회 추상화 
    - 각 클래스에서는 인스턴스 변수로 저장하되, 이 스키마-테이블 구조를 내부에서 받아쓰는 실행 클래스는 스키마와 테이블을 클래스 변수로 모은 뒤 묶어서 사용하기

In [19]:
# from abc import ABC, abstractmethod

class SQLTable(ABC):
    """스키마와 테이블로 이루어진 SQL 테이블명 구조를 저장하고 꺼내쓰는 목적의 추상 클래스"""
    @abstractmethod
    def setSchemaTable(self):
        """스키마명, 테이블명 저장 메서드"""
        pass

    @abstractmethod
    def getSchema(self):
        """스키마명 꺼내는 메서드"""
        pass

    @abstractmethod
    def getTable(self):
        """테이블명 꺼내는 메서드"""
        pass

In [22]:
class TableContainer(SQLTable):
    """스키마, 테이블 입출력 클래스"""
    def __init__(self):
        self._schema = None
        self._table = None

    def setSchemaTable(self, schema: str, table: str):
        """스키마, 테이블 입력 클래스"""
        self._schema = schema
        self._table = table

    def getSchema(self):
        return self._schema
    
    def getTable(self):
        return self._table



In [27]:
class InitTables:
    """원본 스키마&테이블, 결합키 스키마&테이블, 결합대상정보 스키마&테이블 모으기"""
    def __init__(self):
        self.original_table = None
        self.key_table = None
        self.target_table = None
        self.serial_col = None
        self.serial_text = None
        self.key_cols = None

    def addOriginalTable(self, original_table: TableContainer):
        """원본 스키마 & 테이블 입력 메서드"""
        self.original_table = original_table

    def addKeyTable(self, key_table: TableContainer):
        """결합키 스키마 & 테이블 입력 메서드"""
        self.key_table = key_table

    def addTargetTable(self, target_table: TableContainer):
        """결합대상정보 스키마 & 테이블 입력 메서드"""
        self.target_table = target_table

    def addSerialColInfo(self, serial_col: str, serial_text: str):
        """일련번호 컬럼명 (serial_col), 일련번호 내용 (serial_text1, serial_text2 형태) 입력 메서드"""
        self.serial_col = serial_col
        self.serial_text = serial_text

    def addKeyColInfo(self, columns: list):
        """결합키 생성 항목 컬럼명 입력 메서드"""
        self.key_cols = columns


In [28]:
class DivideOriginalTable(PyMySQLQuery):
    """원본 테이블을 결합키와 결합대상정보로 분할하기"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        super().__init__(pw = pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)
        self.init_tables = None
        self.original_table = None
        self.serial_col = None
        self.serial_text = None
        self.key_cols = None
        

    def addInitTables(self, init_tables: InitTables):
        """원본 테이블, 결합키 테이블, 결합대상정보 테이블 객체 통해 입력"""
        self.init_tables = init_tables

        self.original_table = self.init_tables.original_table
        self.serial_col = self.init_tables.serial_col
        self.serial_text = self.init_tables.serial_text

        self.key_cols = self.init_tables.key_cols

    def addSerialNum(self):
        """일련번호 컬럼 추가 메서드"""
        schema = self.original_table.getSchema()
        table = self.original_table.getTable()

        # 컬럼명 중복시 해당 컬럼 삭제
        super().dataQueryLanguage(f"ALTER TABLE {schema}.{table} DROP COLUMN {self.serial_col}")
        super().executeQuery()

        # 컬럼 만들기
        make_column = f"ALTER TABLE {schema}.{table} ADD COLUMN {self.serial_col} VARCHAR(1000)"
        super().dataQueryLanguage(make_column)
        super().executeQuery()

        # 값 할당
        super().dataQueryLanguage("SET @counter = 0;")
        super().executeQuery()
        
        super().dataQueryLanguage(f"UPDATE {schema}.{table} SET {self.serial_col} = CONCAT('{self.serial_text}', @counter := @counter + 1);")
        super().executeQuery()

    def insertKey(self):
        """결합키 테이블 DB 입력 메서드"""
        key_table = self.init_tables.key_table
        key_schema = key_table.getSchema()
        key_result = key_table.getTable()

        join_columns = ', '.join(self.key_cols)

        super().dataQueryLanguage(f"DROP TABLE IF EXISTS {key_schema}.{key_result}")
        super().executeQuery()
        
        sql = f"CREATE TABLE {key_schema}.{key_result} AS SELECT {self.serial_col}, {join_columns} FROM {self.original_table}"
        super().dataQueryLanguage(sql)
        super().executeQuery()

        # 결합키 컬럼 만들기
        super().dataQueryLanguage(f"ALTER TABLE {key_schema}.{key_result} ADD COLUMN {self.key_cols} VARCHAR(1000)")
        super().executeQuery()

        super().dataQueryLanguage(f"UPDATE {key_schema}.{key_result} SET {self.key_cols} = CONCAT({join_columns})")
        super().executeQuery()
        
    def insertTarget(self):
        """결합대상정보 테이블 DB 입력 메서드
           : 원래 테이블을 복사한 뒤, 결합키 생성항목 columns를 제거하여 생성
        """
        target_table = self.init_tables.target_table
        target_schema = target_table.getSchema()
        target_result = target_table.getTable()

        super().dataQueryLanguage(f"DROP TABLE IF EXISTS {target_schema}.{target_result}")
        super().executeQuery()

        create_sql = f"CREATE TABLE {target_schema}.{target_result} AS SELECT * FROM {self.original_table}"
        super().dataQueryLanguage(create_sql)
        super().executeQuery()

        for column in self.key_cols:
            drop_sql = f"ALTER TABLE {target_schema}.{target_table} drop {column}"
            super().dataQueryLanguage(drop_sql)
            super().executeQuery()

In [ ]:
class EncryptKeyCol:
    """결합키 암호화 클래스"""
    def __init__(self):
        pass

In [ ]:
class CreateMappingTable:
    """매핑테이블 만들기 클래스"""
    def __init__(self):
        pass

## 최상위 실행 클래스

In [ ]:
class CryptoGrapher:
    """원본 테이블 입력부터 반출까지 전부 실행하는 최상위 실행 클래스"""
    def __init__(self):
        pass